This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-run-anywhere/edge-architecture-publish-cv-resnet-model).

## Run Anywhere for ARM Architecture Tutorial: Custom Inference Computer Vision with Resnet50

Wallaroo Run Anywhere provides model deployment in any device, any cloud, and any architecture.  Models uploaded to Wallaroo are set to their targeted architecture.

Organizations can deploy uploaded models to clusters that have nodes with the provisioned architecture.  The following architectures are supported:

* `X86`:  The standard X86 architecture.
* `ARM`:  For more details on cloud providers and their ARM offerings, see [Create ARM Nodepools for Kubernetes Clusters](https://staging.docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-install/wallaroo-install-enterprise-environment/wallaroo-arm-nodepools/).

### Model Architecture Inheritance

The model's deployment configuration inherits its architecture.  Models automatically deploy in the target architecture provided nodepools with the architecture are available.  For information on setting up nodepools with specific architectures, see [Infrastructure Configuration Guides](https://staging.docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-install/wallaroo-install-enterprise-environment/).

That deployment configuration is carried over to the models' publication in an Open Container Initiative (OCI) Registries, which allows edge model deployments on `X64` and `ARM` architectures.  More details on deploying models on edge devices is available with the [Wallaroo Run Anywhere Guides](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

The deployment configuration **can** be overridden for either model deployment in the Wallaroo Ops instance, or in the Edge devices.

This tutorial demonstrates deploying a ML model trained to predict house prices to ARM edge locations through the following steps.

* Upload a model with the architecture set to `ARM`.
* Create a pipeline with the uploaded model as a model step.
* Publish the pipeline model to an Open Container Initiative (OCI) Registry for both X64 and ARM deployments.

In this notebook, we use a Computer Vision Resnet-50 model wrapped in a BYOP container to add additional enhancements.  For more information on custom inference models using BYOP (Bring Your Own Predict), see [Model Uploads and Registrations: Arbitrary Python](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/).

## Goal

Demonstrate publishing a pipeline with model steps to various architectures.

### Resources

This tutorial provides the following:

* Models:
  * `model-with-pixel-intensity.zip`: A custom inference model combined with the Resnet50 model.  This is downloaded from the Wallaroo Public Data bucket from the following URL.  Unzip the following .zip file into the `./models` folder:
    * [https://storage.googleapis.com/wallaroo-public-data/cv-demo-models/cv-retail-models.zip](https://storage.googleapis.com/wallaroo-public-data/cv-demo-models/cv-retail-models.zip)

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * `json`: Used for format input data for inference requests.
* A X64 Docker deployment to deploy the model on an edge location.


## Model Upload and Pipeline Steps

The following process will prepare for the pipeline publish.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
from wallaroo.engine_config import Architecture
import pyarrow as pa

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime
import time

workspace_name = f'run-anywhere-architecture-cv-byop'
arm_pipeline_name = f'architecture-demonstration-arm'
model_name_arm = f'computer-vision-with-pixel-intensity-arm'
model_file_name = './models/model-with-pixel-intensity.zip'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace, pipeline, and model names should be unique to each user, so we'll add in a randomly generated suffix so multiple people can run this tutorial in a Wallaroo instance without effecting each other.

In [4]:
workspace = get_workspace(workspace_name, wl)

wl.set_current_workspace(workspace)

{'name': 'run-anywhere-architecture-cv-byop', 'id': 2900924, 'archived': False, 'created_by': 'b4a9aa3d-83fc-407a-b4eb-37796e96f1ef', 'created_at': '2024-04-01T18:11:09.314726+00:00', 'models': [{'name': 'computer-vision-with-pixel-intensity-arm', 'versions': 5, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 1, 18, 33, 11, 594492, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 1, 18, 11, 17, 576638, tzinfo=tzutc())}], 'pipelines': []}

### Upload Models and Set ARM Target Architecture

For our example, we will upload the champion model that has been trained to detect objects from a photo.  The BYOP version outputs additional fields including the average confidence and pixel intensity.

Models are uploaded to Wallaroo via the [`wallaroo.client.upload_model`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/client/#Client.upload_model) method which takes the following arguments:

| Parameter | Type | Description |
|---|---|---|
| **path** | *String* (*Required*) | The file path to the model. |
| **framework** | *wallaroo.framework.Framework* (*Required*) | The model's framework.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for supported model frameworks. |
| **input_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's input schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **output_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's output schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **convert_wait** | *bool* (*Optional*)  | Whether to wait in the SDK session to complete the auto-packaging process for non-native Wallaroo frameworks. |
| **arch** | *wallaroo.engine_config.Architecture* (*Optional*)  | The targeted architecture for the model.  Options are <ol><li>`X86` (*Default*)</li><li>`ARM`</li></ol> |

We upload the model and set the architecture to `ARM`.

In [ ]:
input_schema = pa.schema([
    pa.field('tensor', pa.list_(
        pa.list_(
            pa.list_(
                pa.float32(), # images are normalized
                list_size=640
            ),
            list_size=480
        ),
        list_size=3
    )),
])

output_schema = pa.schema([
    pa.field('boxes', pa.list_(pa.list_(pa.float32(), list_size=4))),
    pa.field('classes', pa.list_(pa.int64())),
    pa.field('confidences', pa.list_(pa.float32())),
    pa.field('avg_px_intensity', pa.list_(pa.float32())),
    pa.field('avg_confidence', pa.list_(pa.float32())),
])


model_name_arm = f'computer-vision-with-pixel-intensity-arm'
model_file_name = './models/model-with-pixel-intensity.zip'

arm_model = wl.upload_model(model_name_arm, 
                        model_file_name, 
                        framework=Framework.CUSTOM,
                        input_schema=input_schema, 
                        output_schema=output_schema,
                        arch=Architecture.ARM)

In [11]:
display(arm_model)

Name,computer-vision-with-pixel-intensity-arm
Version,e5c03a33-b741-4bad-90b0-4ff0ba972a27
File Name,model-with-pixel-intensity.zip
SHA,6d58039b1a02c5cce85646292965d29056deabdfcc6b18c34adf566922c212b0
Status,error
Image Path,None
Architecture,arm
Acceleration,none
Updated At,2024-01-Apr 18:34:56


### Build Pipeline

We build the pipeline with the `wallaroo.client.build_pipeline(pipeline_name` command, and set the model as a model step in the pipeline.

In [12]:
pipeline_arm = wl.build_pipeline('architecture-demonstration-arm')
pipeline_arm.add_model_step(arm_model)

name,architecture-demonstration-arm
created,2024-04-01 18:36:26.347071+00:00
last_updated,2024-04-01 18:36:26.347071+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,aa86387c-813a-40de-b07a-baf388e20d67
steps,
published,False


## Pipeline Publish for ARM Architecture via the Wallaroo SDK

We now publish our pipeline as two different versions.

* `ARM`:  The model's architecture was set to `ARM`, when when we publish the pipeline to the OCI registry, it will automatically inherit that architecture.
* `X64`:  We **override** the model's architecture to push a published version that can be deployed on `X64` based devices.

### Publish Pipeline for ARM

Publishing the pipeline uses the pipeline `wallaroo.pipeline.Pipeline.publish()` command.  This requires that the Wallaroo Ops instance have [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) enabled.

When publishing, we specify the pipeline deployment configuration through the `wallaroo.DeploymentConnfigBuilder` and specify the architecture as `wallaroo.engine_config.Architecture.ARM`.

The following publishes the pipeline to the OCI registry and displays the container details.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/).

In [13]:
deploy_config_arm = (wallaroo.DeploymentConfigBuilder()
                     .replica_count(1)
                     .cpus(1)
                     .memory("1Gi")
                     .arch(wallaroo.engine_config.Architecture.ARM)
                     .build()
                    )
assay_pub_arm = pipeline_arm.publish(deployment_config=deploy_config_arm)
assay_pub_arm

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing......... Published.


ID,64
Pipeline Name,architecture-demonstration-arm
Pipeline Version,fe2b6f05-3623-4440-8258-9e5828bc7eaf
Status,Published
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini-aarch64:v2024.1.0-main-4833
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/architecture-demonstration-arm:fe2b6f05-3623-4440-8258-9e5828bc7eaf
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/architecture-demonstration-arm
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:9d6baf0453aadd94026b730b84577ce52fa1b5ced01993ec53b14d8d913ef10b
Helm Chart Version,0.0.1-fe2b6f05-3623-4440-8258-9e5828bc7eaf
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'arm', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


### Publish Pipeline for X86

We will publish the pipeline again, this time by overriding the deployment architecture and setting it to `ARM`.  Note the changes to the Engine URL, and the `Docker Run Command` and `Helm Install Commands` that reflect the changes.

We change the deployment architecture with the `wallaroo.DeploymentConfigBuilder` object, changing the `arch` setting to `X86`.

In [14]:
deploy_config_x86 = (wallaroo.DeploymentConfigBuilder()
                     .replica_count(1)
                     .cpus(1)
                     .memory("1Gi")
                     .arch(wallaroo.engine_config.Architecture.X86)
                     .build()
                    )
assay_pub_x86 = pipeline_arm.publish(deployment_config = deploy_config_x86)
assay_pub_x86

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing......... Published.


ID,65
Pipeline Name,architecture-demonstration-arm
Pipeline Version,ef9e2394-e29f-46dc-aaa4-eda0a304a71e
Status,Published
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4833
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/architecture-demonstration-arm:ef9e2394-e29f-46dc-aaa4-eda0a304a71e
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/architecture-demonstration-arm
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:764641a568575f29209e1348c55aa68eb61197396dc689b8bcc6f54ce2149a9f
Helm Chart Version,0.0.1-ef9e2394-e29f-46dc-aaa4-eda0a304a71e
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


For details on performing inference requests through an edge deployed model, see [Edge Deployment Endpoints](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/wallaroo-model-operations-run-anywhere-deploy/#edge-deployment-endpoints).